In [1]:
# export CUDA_VISIBLE_DEVICES=2,5
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import pkg_resources
from datasets.downstream import uet175_dataset
from finetune_evaluator import Evaluator
from downstream import model_for_uet175
import torch
from torch import nn
from torchsummary import summary

/tmp/ipykernel_1890798/2827869089.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
def count_parameters(module: nn.Module) -> None:
    """
    Print the number of total and trainable parameters in a torch.nn.Module.
    """
    total_params = sum(p.numel() for p in module.parameters())
    trainable_params = sum(p.numel() for p in module.parameters() if p.requires_grad)

    print(f"Total parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")

In [8]:
class Params:
    def __init__(self):
        self.datasets_dir = "/mnt/disk1/aiotlab/namth/EEGFoundationModel/datasets/UET175_eegpt_108"
        self.batch_size = 8
        self.use_pretrained_weights = True
        self.foundation_dir = "/mnt/disk1/aiotlab/namth/EEGFoundationModel/EEGPT/checkpoint/108.ckpt"
        self.dropout = 0.1
        self.cuda = 2
        self.modality_mode = 'multi'
        self.project_name = 'BigFaced'
        self.frozen = True
        self.num_of_classes = 4
        self.num_fold = 0

params = Params()

loader = uet175_dataset.LoadDataset(params).get_data_loader()

train_eval = Evaluator(params, loader['train'])
val_eval = Evaluator(params, loader['val'])
test_eval = Evaluator(params, loader['test'])

1260 335 286
1881


In [9]:
x, y = next(iter(loader['train']))
print(x.shape, y.shape)
y.max()

torch.Size([8, 22, 800]) torch.Size([8])


tensor(3)

In [10]:
model = model_for_uet175.LitEEGPTCausal(params)

In [11]:
model(x).shape

torch.Size([8, 4])